In [91]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import math
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

C:\Users\DELL\AppData\Local\Temp\ipykernel_1800\762098501.py:3: DeprecationWarning:

Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display



In [66]:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [9]:
 # confirmed_df.head()

In [10]:
 # recovered_df.head()

In [11]:
# death_df.head()

In [12]:
# country_df.head()

In [74]:
# data cleaning
# renaming the df columns to lowercase
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

# changing province/state to state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = recovered_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})



In [75]:
country_df

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3,cases_28_days,deaths_28_days
0,Afghanistan,2022-07-20 05:20:41,33.939110,67.709953,183908,7736,NaN,NaN,472.427017,NaN,NaN,4.206451,4,AFG,2100,21
1,Albania,2022-07-20 05:20:41,41.153300,20.168300,293917,3517,NaN,NaN,10213.253180,NaN,NaN,1.196596,8,ALB,16254,20
2,Algeria,2022-07-20 05:20:41,28.033900,1.659600,266542,6875,NaN,NaN,607.835029,NaN,NaN,2.579331,12,DZA,549,0
3,Andorra,2022-07-20 05:20:41,42.506300,1.521800,45061,153,NaN,NaN,58320.067301,NaN,NaN,0.339540,20,AND,1612,0
4,Angola,2022-07-20 05:20:41,-11.202700,17.873900,102209,1912,NaN,NaN,310.984502,NaN,NaN,1.870677,24,AGO,2448,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,West Bank and Gaza,2022-07-20 05:20:41,31.952200,35.233200,672039,5664,NaN,NaN,13173.577689,NaN,NaN,0.842808,275,PSE,13536,4
195,Winter Olympics 2022,2022-07-20 05:20:41,39.904200,116.407400,535,0,NaN,NaN,NaN,NaN,NaN,0.000000,2022,NaN,0,0
196,Yemen,2022-07-20 05:20:41,15.552727,48.516388,11849,2149,NaN,NaN,39.727126,NaN,NaN,18.136552,887,YEM,25,0
197,Zambia,2022-07-20 05:20:41,-13.133897,27.849332,328550,4013,NaN,NaN,1787.156149,NaN,NaN,1.221427,894,ZMB,3992,15


In [76]:
# total number of confirmed , death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())



In [77]:
# displaying the total stats
display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: #fff; font-size:30px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )

In [78]:
# sorting the values by confirmed descending order
# country_df.sort_values('confirmed', ascending= False).head(10).style.background_gradient(cmap='copper')
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    y = 'background-color: purple'
    g = 'background-color: grey'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = y
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    
    return df1

    def  show_latest_cases(n):
        n = int(n)
        return country_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)
    
    interact(show_latest_cases, n='10')
    
    ipywLayout = widgets.Layout(border=' solid 2px green')
    ipywLayout.display='none'
    widgets.VBox([fig], layout=ipywLayout)
    
    
    
    
    
    
    

In [79]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)

In [80]:
# # plotting the 20 worst hit countries
def bubble_chart(n):
    fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country", hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 700
    )
    fig.show();
    
    interact(bubble_chart, n=10)
    
    ipywLayout = widgets.Layout(border='solid 2px green')
    ipywLayout.display='none'
    widgets.VBox([fig], layout=ipywLayout)
    

In [81]:
def plot_cases_of_a_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6,8]
    line_size = [4,5]
    
    df_list = [confirmed_df, death_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country =='world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
    
    else:
        x_data = np.array(list(df.iloc[:, 20:].columns))
        y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
    print(i)
    fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
                            name=labels[i],
                            line=dict(color=colors[i], width=line_size[i]),
                            connectgaps=True,
                            text = "Total" + str(labels[i]) +": "+ str(y_data[-1])
                            ));
    
    fig.update_layout(title="COVID 19 cases of " + country,
    xaxis_title='Date',
    yaxis_title='No. of Confirmed Cases',
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="lightgrey",
    width = 800,
     
    );
    
    fig.update_yaxes(type="linear")
    fig.show();

In [82]:
interact(plot_cases_of_a_country, country='World')

ipywLayout = widgets.Layout(borders='solid 2px green')
ipywLayout.display='none' #uncomment this, runcell again - then the graph disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Text(value='World', description='country'), Output()), _dom_classes=('widget-interact',)…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(display='none'…

In [86]:
sorted_country_df

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3,cases_28_days,deaths_28_days
184,US,2022-07-20 05:20:41,40.000000,-100.000000,89830640,1024858,NaN,NaN,27265.503220,NaN,NaN,1.140878,840,USA,3286350,10610
80,India,2022-07-20 05:20:41,20.593684,78.962880,43801678,525821,NaN,NaN,3174.024552,NaN,NaN,1.200459,356,IND,451417,882
24,Brazil,2022-07-20 05:20:41,-14.235000,-51.925300,33397814,675871,NaN,NaN,15712.225658,NaN,NaN,2.023698,76,BRA,1578987,6481
63,France,2022-07-20 05:20:41,46.227600,2.213700,33375449,152019,NaN,NaN,51150.236484,NaN,NaN,0.455482,250,FRA,2898534,1818
67,Germany,2022-07-20 05:20:41,51.165691,10.451526,29994679,142771,NaN,NaN,36070.792878,NaN,NaN,0.475988,276,DEU,2540454,2309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,Micronesia,2022-07-20 05:20:41,7.425600,150.550800,38,0,NaN,NaN,33.387515,NaN,NaN,0.000000,583,FSM,0,0
76,Holy See,2022-07-20 05:20:41,41.902900,12.453400,29,0,NaN,NaN,3584.672435,NaN,NaN,0.000000,336,VAT,0,0
5,Antarctica,2022-07-20 05:20:41,-71.949900,23.347000,11,0,NaN,NaN,NaN,NaN,NaN,0.000000,10,ATA,0,0
107,MS Zaandam,2022-07-20 05:20:41,NaN,NaN,9,2,NaN,NaN,NaN,NaN,NaN,22.222222,8888,NaN,0,0


In [84]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "confirmed",
    title= "Top 10 worst affected countries", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

In [88]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "deaths",
    title= "Top 10 worst affected countries", # the axis names
    color_discrete_sequence=["dimgray"], 
    height=500,
    width=800
)


In [93]:
world_map = folium.Map(location=[25,0], tiles="cartodbpositron", zoom_start = 2, max_zoom = 5, min_zoom = 2, max_bounds=True)


for i in range(0, len(confirmed_df)):
    if not math.isnan(float(confirmed_df.iloc[i]['lat'])):
        folium.Circle(fill=True, location=(confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']),
                      width='100%', height='100%', left='0%', top='0%', position='relative',
                      radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*30000,
                      color="#FF4500",
                      fill_color='indigo',
                      tooltip = "<div style='margin: 0; background-color: dimgray; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
                     ).add_to(world_map)

world_map